In [1]:
import pandas as pd
import numpy as np

## Prepare FiveThirtyEight Data

In [2]:
df1 = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/comic-characters/dc-wikia-data.csv")
df1['publisher'] = "DC Comics"
df1.rename(str.lower, axis='columns', inplace=True)
df1.rename(columns={'id':'identity_type', 'name':'character_name'}, inplace=True)

In [3]:
df2 = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/comic-characters/marvel-wikia-data.csv")
df2['publisher'] = "MARVEL Comics"
df2.rename(str.lower, axis='columns', inplace=True)
df2.rename(columns={'id':'identity_type', 'name':'character_name'}, inplace=True)

In [4]:
df_fte = pd.concat([df1, df2], ignore_index=True)

In [5]:
df_fte.head()

,page_id,character_name,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0,DC Comics
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0,DC Comics
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0,DC Comics
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0,DC Comics
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0,DC Comics


In [6]:
# ss = df_fte.duplicated(subset=['character_name', 'first appearance','year'], keep=False)
# df_fte[ss==True].sort_values(by=['character_name'])

In [7]:
# df_fte[(df_fte.character_name == 'charlie') & (df_fte.year == 1963.0)]

#### Making join column 

In [8]:
def get_year_month(first_appearance):  # 1939, May
    datetime = pd.to_datetime(first_appearance, errors='ignore', yearfirst=True) 
    try:
        dt_str = datetime.strftime('%b-%y')  # e.g. 1939-05
        return dt_str
    except ValueError:
        return np.nan
    except AttributeError:
        if datetime == "1988, Holiday":
            datetime = "1988, December"  # I guess...
            datetime = pd.to_datetime(datetime, errors='ignore', yearfirst=True)  
            dt_str = datetime.strftime('%b-%y')  # e.g. 1939-05
            return dt_str
        if type(datetime) == str:
            datetime = pd.to_datetime(datetime, errors='ignore', yearfirst=True) 
            return datetime
        return datetime

In [9]:
df_fte['first appearance'] = df_fte.apply(lambda row: get_year_month(row['first appearance']), axis=1)

In [10]:
df_fte['join_name'] = df_fte.apply(lambda row: f"{row.character_name.split('(')[0].strip().replace(' ', '_').lower()}_{str(row['first appearance']).replace('-', '_').lower()}", axis=1)

In [11]:
df_fte.shape

(23272, 15)

In [12]:

df_fte.head()

,page_id,character_name,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher,join_name
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,May-39,1939.0,DC Comics,batman_may_39
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,Oct-86,1986.0,DC Comics,superman_oct_86
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,Oct-59,1959.0,DC Comics,green_lantern_oct_59
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,Feb-87,1987.0,DC Comics,james_gordon_feb_87
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,Apr-40,1940.0,DC Comics,richard_grayson_apr_40


In [13]:
df_fte.join_name.value_counts()

michael_van_patrick_aug_07       3
ghost_rider_may_09               3
peter_parker_mar_95              3
squid_feb_89                     3
hoenir_sep_76                    2
kerwin_kronus_oct_76             2
tomazooma_nov_68                 2
charlie_cluster-7_feb_13         2
fuel_nan                         2
clown_aug_09                     2
dave_nan                         2
jason_nan                        2
agent_x_mar_42                   2
caretaker_oct_08                 2
spymaster_jan_71                 2
gemini_jul_77                    2
branden_mar_87                   2
lawrence_zaxton_aug_63           2
laralei_aug_77                   2
titannus_jan_05                  2
angel_of_death_apr_93            2
devil_sep_81                     2
frankenstein's_monster_jan_68    2
thelius_nov_75                   2
carbon_oct_79                    2
angel_jul_96                     2
james_madrox_may_87              2
kerberos_jan_06                  2
reno_aug_99         

This repetition is here since the characters are e.g. changing align (going from good to bad), or have died, etc.

In [14]:
df_fte[df_fte.join_name=='superman_Oct-86']

,page_id,character_name,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher,join_name


In [15]:
df_fte.drop(['character_name'], axis=1, inplace=True)

---

## Prepare ComicVine Data

In [16]:
def extract_origin_name(x):
    origin = x.get('origin', None)
    origin_name = origin.get('name', origin) if origin else x
    return origin_name if type(origin_name) == str else np.nan

In [17]:
def extract_issue_cover_date(x):
    first_appeared_in_issue = x.get('first_appeared_in_issue', None)
    issue_id = first_appeared_in_issue.get('id', None) if first_appeared_in_issue else np.nan
    return df_is[df_is.issue_id == issue_id].cover_date.values[0] if issue_id and first_appeared_in_issue else np.nan

In [18]:
df_characters = pd.read_json("../ComicVine/characters.json")
df_cv = df_characters[['name', 'birth', 'count_of_issue_appearances','aliases', 'deck', 'description', 'id', 'image', 'origin', 'real_name', 'first_appeared_in_issue']]
df_cv.rename(columns={'id':'character_id', 'name':'character_name'}, inplace=True)

df_issues = pd.read_json("../ComicVine/issues.json")
df_is = df_issues[['id', 'cover_date']]
df_is.rename(columns={'id':'issue_id'}, inplace=True)


C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [19]:
df_cv['image'] = df_cv.apply(lambda row: row.image['super_url'], axis=1)
df_cv['origin'] = df_cv.apply(lambda row: extract_origin_name(row), axis=1)
df_cv['first_appeared_in_issue'] = df_cv.apply(lambda row: extract_issue_cover_date(row), axis=1)
df_cv['first_appeared_in_issue'] = df_cv.apply(lambda row: get_year_month(row['first_appeared_in_issue']), axis=1)

C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [20]:
df_cv.head()

,character_name,birth,count_of_issue_appearances,aliases,deck,description,character_id,image,origin,real_name,first_appeared_in_issue
0,Lightning Lad,None,903,Garth Ranzz\nLightning Boy\nProty\nLive Wire\n...,Garth Ranzz is one of the three original membe...,"<h2>Origin</h2><figure data-align=""right"" data...",1253,https://comicvine.gamespot.com/api/image/scale...,Alien,Garth Ranzz,Apr-58
1,Dream Girl,None,385,Nura Nal\nDreamy\nDreamer\nMiss Terious\nNura ...,"Nura Nal is from the planet Naltor, where all ...","<h2>Origin</h2><figure data-align=""right"" data...",1254,https://comicvine.gamespot.com/api/image/scale...,Alien,Nura Nal,Feb-64
2,Brainiac 5,None,977,Querl Dox\nBrainiac 5.1\nBrainy\nBrainiac \nB-...,Brainiac 5 is a long-standing member of the Le...,<h2>Current Events</h2><p>Brainiac 5 is presen...,1255,https://comicvine.gamespot.com/api/image/scale...,Alien,Querl Dox,May-61
3,Invisible Kid,None,387,Lyle Norg\nBrainiac 6\nInvisible Kid,"He is a member of the Legion of Super-Heroes, ...","<p data-right-indent=""0"" data-left-indent=""0"">...",1256,https://comicvine.gamespot.com/api/image/scale...,Human,Lyle Norg,Aug-60
4,Phantom Girl,None,654,Tinya Wazzo\nTinya Wazzo-Nah\nPhantom Woman\nP...,Tinya Wazzo is a native of the extra-dimension...,<h2>Current Events</h2><p>Phantom Girl current...,1257,https://comicvine.gamespot.com/api/image/scale...,Alien,Tinya Wazzo,May-61


#### Making join column 

In [21]:
df_cv['join_name'] = df_cv.apply(lambda row: f"{row.character_name.strip().replace(' ', '_').lower()}_{str(row['first_appeared_in_issue']).replace('-', '_').lower()}", axis=1)

C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_cv.join_name.value_counts()

dupe_nan                     28
emma_jun_16                   4
tiger_nan                     3
joe_jul_15                    3
medusa_nan                    3
nick_oct_18                   3
liz_may_15                    3
silas_jul_15                  3
shawn_nov_15                  3
sarah_jul_14                  3
john_apr_14                   3
tom_may_17                    3
adam_may_18                   3
mara_nan                      3
claire_mar_15                 3
sarah_apr_15                  3
sam_mar_15                    3
max_nov_13                    3
franco_apr_14                 3
tom_mar_15                    3
alex_apr_15                   3
kokoro_dec_14                 3
evan_jun_14                   3
dave_oct_14                   3
eva_nan                       3
jack_mar_16                   3
larry_nan                     3
mike_dec_13                   3
alex_aug_14                   3
finn_dec_15                   3
                             ..
bron_wox

## Joining data

In [23]:
df = pd.merge(df_fte, df_cv, on='join_name');

In [24]:
df.shape

(7408, 25)

In [25]:
df.head(2)

,page_id,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,...,birth,count_of_issue_appearances,aliases,deck,description,character_id,image,origin,real_name,first_appeared_in_issue
0,1422,\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,...,"Feb 19, 1972",16603,Bruce Wayne\r\nThe Caped Crusader\r\nThe Dark ...,"Bruce Wayne, who witnessed the murder of his m...","<h2>Current Events</h2><ul><li><a href=""https:...",1699,https://comicvine.gamespot.com/api/image/scale...,Human,Bruce Wayne,May-39
1,1448,\/wiki\/Wonder_Woman_(Diana_Prince),Public Identity,Good Characters,Blue Eyes,Black Hair,Female Characters,NaN,Living Characters,1231.0,...,"Mar 21, 1881",6575,Diana Prince\r\nDiana of Themyscira\r\nPrinces...,"The princess of the Amazons, armed with powers...",<h2><strong>Origin</strong></h2><figure data-a...,2048,https://comicvine.gamespot.com/api/image/scale...,Other,Diana of Themyscira,Dec-41


In [26]:
df.columns

Index(['page_id', 'urlslug', 'identity_type', 'align', 'eye', 'hair', 'sex',
       'gsm', 'alive', 'appearances', 'first appearance', 'year', 'publisher',
       'join_name', 'character_name', 'birth', 'count_of_issue_appearances',
       'aliases', 'deck', 'description', 'character_id', 'image', 'origin',
       'real_name', 'first_appeared_in_issue'],
      dtype='object')

### Checking whether tables are joined correctly

In [27]:
df.join_name.value_counts()

cole_may_04                  2
carbon_oct_79                2
kerberos_jan_06              2
tomazooma_nov_68             2
victoria_nan                 2
kerwin_kronus_oct_76         2
komodo_nov_95                2
michael_nan                  2
branden_mar_87               2
spymaster_jan_71             2
micron_aug_10                2
titannus_jan_05              2
thunder_nan                  2
thelius_nov_75               2
lorelei_nov_83               2
link_jun_94                  2
harmony_aug_01               2
josef_muller_sep_06          1
gretti_may_83                1
wesley_fermin_feb_87         1
tyrak_dec_76                 1
fence_oct_12                 1
abel_stack_jul_77            1
troubleshooter_apr_93        1
kevin_kennerman_nov_73       1
agrom_oct_89                 1
mindmeld_jan_97              1
gullin_apr_64                1
max_belmonde_apr_94          1
prodigal_aug_92              1
                            ..
gpaak_aug_93                 1
sidewind

Check the Superman, there is 2 of them:

In [28]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 2999

In [29]:
df_fte[df_fte.join_name.str.startswith("superman") == 1]

,page_id,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher,join_name
1,23387,\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,Oct-86,1986.0,DC Comics,superman_oct_86


In [30]:
df_cv[df_cv.join_name.str.startswith("superman") == 1]

,character_name,birth,count_of_issue_appearances,aliases,deck,description,character_id,image,origin,real_name,first_appeared_in_issue,join_name
379,Superman,"Jun 18, 1971",14663,Kal-El\r\nClark Kent\r\nClark Jerome Kent\r\nC...,Rocketed to Earth as an infant from the doomed...,"<h2>Current Events</h2><figure data-align=""rig...",1807,https://comicvine.gamespot.com/api/image/scale...,Alien,Kal-El / Clark Joseph Kent,Jun-38,superman_jun_38
5928,Superman Robot,None,31,Power Man\nAjax\nAdam Newman\nMacDuff\nNed,"Ned, the last robotic replica of Superman, he ...",<h2>History</h2><p>One of the new Robot Superm...,9576,https://comicvine.gamespot.com/api/image/scale...,Robot,"""Ned""",Oct-59,superman_robot_oct_59
29455,Superman,None,35,The Superman\r\nTangent Superman (Earth-9)\r\n...,Harvey Dent has evolved into the most powerful...,"<h2>Origin</h2><figure data-align=""right"" data...",45116,https://comicvine.gamespot.com/api/image/scale...,Other,Harvey Dent,Sep-98,superman_sep_98
35501,Superman Junior,None,28,Superman Jr.\nSuper Sons,"In some story lines, Superman had a son called...",<h2>History</h2><p>Superman Junior is <a href=...,55007,https://comicvine.gamespot.com/api/image/scale...,Human,None,Dec-65,superman_junior_dec_65
36796,Superman Predator,None,1,None,Superman Predator is one of the Metapredators....,None,56708,https://comicvine.gamespot.com/api/image/scale...,Alien,None,Aug-01,superman_predator_aug_01
92304,Superman (earth-D),None,4,None,Superman of earth-D and husband of Supergirl.,<p>The Superman of earth D died fighting the S...,115385,https://comicvine.gamespot.com/api/image/scale...,Alien,Kal-El,Feb-99,superman_(earth-d)_feb_99
122511,Superman-X,None,1,None,Kryptonite-powered clone of Superman in the 41...,"<p>Superman-X was a clone of <a href=""/superma...",146555,https://comicvine.gamespot.com/api/image/scale...,Alien,Kell-El,Jan-09,superman-x_jan_09


These are two different supermans: [First One](https://comicvine.gamespot.com/character/4005-1807/) and [Second One](https://comicvine.gamespot.com/character/4005-45116/).

Now, check Sam, there is 280 of them:

In [31]:
df[df.join_name.str.startswith("batman") == 1]

,page_id,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher,join_name,character_name,birth,count_of_issue_appearances,aliases,deck,description,character_id,image,origin,real_name,first_appeared_in_issue
0,1422,\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,May-39,1939.0,DC Comics,batman_may_39,Batman,"Feb 19, 1972",16603,Bruce Wayne\r\nThe Caped Crusader\r\nThe Dark ...,"Bruce Wayne, who witnessed the murder of his m...","<h2>Current Events</h2><ul><li><a href=""https:...",1699,https://comicvine.gamespot.com/api/image/scale...,Human,Bruce Wayne,May-39


## Adding List of Powers

In [32]:
import requests

def collect(field):
    def collect_field(character_id): 
        with requests.Session() as s:
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
            api_key = "22b116612a0474be3f74aa7edc1c580f82081b1d"
            r = s.get(f"https://comicvine.gamespot.com/api/character/4005-{character_id}/?api_key={api_key}&field_list={field}&format=json", headers=headers).json()
            try:
                ret_val = list(map(lambda x: x['name'], r['results'][field]))
            except Exception as e:
                print(e)
                print(character_id)
                ret_val = np.nan
            return ret_val
    return collect_field

In [33]:
collect('powers')(1699)

['Agility',
 'Stamina',
 'Intellect',
 'Weapon Master',
 'Insanely Rich',
 'Power Suit',
 'Unarmed Combat',
 'Gadgets',
 'Escape Artist',
 'Tracking',
 'Adaptive',
 'Swordsmanship',
 'Marksmanship',
 'Energy Based Constructs',
 'Stealth',
 'Leadership']

In [34]:
collect('teams')(1699)

['Batman and Robin',
 'Batman Family',
 'Batman Inc.',
 'Batmen of All Nations',
 'Injustice League',
 'Insurgency',
 'Justice League 3000',
 'Justice League Dark',
 'Justice League International',
 'Justice League of America',
 'Justice League Unlimited',
 'Justice Lords',
 'Justice Society of America',
 'Miagani',
 'Mystery Analysts',
 'New Gods',
 'Outsiders',
 'Quinntets',
 'Red Hood Gang',
 'Red Lantern Corps',
 'Sinestro Corps',
 'Sons of the Batman',
 'Suicide Squad',
 'Super Powers',
 'Superman/Batman',
 'Team Entropy',
 'Wayne Enterprises',
 'Wayne Family',
 'White Lantern Corps',
 'Zhuguan']

In [35]:
df['powers'] = df.apply(lambda row: collect('powers')(row['character_id']), axis=1)

list indices must be integers or slices, not str
46381
list indices must be integers or slices, not str
116016
list indices must be integers or slices, not str
9535
list indices must be integers or slices, not str
37309
list indices must be integers or slices, not str
116120
list indices must be integers or slices, not str
71015
list indices must be integers or slices, not str
23513
list indices must be integers or slices, not str
93117
list indices must be integers or slices, not str
75335
list indices must be integers or slices, not str
27208
list indices must be integers or slices, not str
32322
list indices must be integers or slices, not str
38460
list indices must be integers or slices, not str
80615
list indices must be integers or slices, not str
56812
list indices must be integers or slices, not str
2038
list indices must be integers or slices, not str
25798
list indices must be integers or slices, not str
9324
list indices must be integers or slices, not str
2397
list indices

list indices must be integers or slices, not str
46957
list indices must be integers or slices, not str
71016
list indices must be integers or slices, not str
61773
list indices must be integers or slices, not str
46460
list indices must be integers or slices, not str
46848
list indices must be integers or slices, not str
61298
list indices must be integers or slices, not str
126796
list indices must be integers or slices, not str
79253
list indices must be integers or slices, not str
68443
list indices must be integers or slices, not str
58804
list indices must be integers or slices, not str
56377
list indices must be integers or slices, not str
59610
list indices must be integers or slices, not str
78150
list indices must be integers or slices, not str
51264
list indices must be integers or slices, not str
4882
list indices must be integers or slices, not str
7100
list indices must be integers or slices, not str
71306
list indices must be integers or slices, not str
8150
list indices

list indices must be integers or slices, not str
124492
list indices must be integers or slices, not str
58802
list indices must be integers or slices, not str
6841
list indices must be integers or slices, not str
46790
list indices must be integers or slices, not str
81651
list indices must be integers or slices, not str
71460
list indices must be integers or slices, not str
6857
list indices must be integers or slices, not str
46783
list indices must be integers or slices, not str
46784
list indices must be integers or slices, not str
12161
list indices must be integers or slices, not str
79486
list indices must be integers or slices, not str
55832
list indices must be integers or slices, not str
80742
list indices must be integers or slices, not str
2367
list indices must be integers or slices, not str
82083
list indices must be integers or slices, not str
48261
list indices must be integers or slices, not str
6526
list indices must be integers or slices, not str
22816
list indices 

list indices must be integers or slices, not str
72303
list indices must be integers or slices, not str
64811
list indices must be integers or slices, not str
57089
list indices must be integers or slices, not str
79278
list indices must be integers or slices, not str
49700
list indices must be integers or slices, not str
50025
list indices must be integers or slices, not str
69232
list indices must be integers or slices, not str
48933
list indices must be integers or slices, not str
49692
list indices must be integers or slices, not str
50289
list indices must be integers or slices, not str
71018
list indices must be integers or slices, not str
56933
list indices must be integers or slices, not str
17655
list indices must be integers or slices, not str
66744
list indices must be integers or slices, not str
6138
list indices must be integers or slices, not str
72262
list indices must be integers or slices, not str
87670
list indices must be integers or slices, not str
72258
list indice

list indices must be integers or slices, not str
117230
list indices must be integers or slices, not str
67905
list indices must be integers or slices, not str
69434
list indices must be integers or slices, not str
67925
list indices must be integers or slices, not str
78634
list indices must be integers or slices, not str
71411
list indices must be integers or slices, not str
68462
list indices must be integers or slices, not str
64812
list indices must be integers or slices, not str
62667
list indices must be integers or slices, not str
64044
list indices must be integers or slices, not str
66557
list indices must be integers or slices, not str
58707
list indices must be integers or slices, not str
62668
list indices must be integers or slices, not str
62670
list indices must be integers or slices, not str
70674
list indices must be integers or slices, not str
62025
list indices must be integers or slices, not str
66556
list indices must be integers or slices, not str
61680
list indi

...

list indices must be integers or slices, not str
87036
list indices must be integers or slices, not str
76439
list indices must be integers or slices, not str
76438
list indices must be integers or slices, not str
61775
list indices must be integers or slices, not str
27547
list indices must be integers or slices, not str
27549
list indices must be integers or slices, not str
10237
list indices must be integers or slices, not str
53051
list indices must be integers or slices, not str
27500
list indices must be integers or slices, not str
52948
list indices must be integers or slices, not str
22065
list indices must be integers or slices, not str
15671
list indices must be integers or slices, not str
54105
list indices must be integers or slices, not str
60086
list indices must be integers or slices, not str
92607
list indices must be integers or slices, not str
57371
list indices must be integers or slices, not str
92610
list indices must be integers or slices, not str
48072
list indic

list indices must be integers or slices, not str
19389
list indices must be integers or slices, not str
25066
list indices must be integers or slices, not str
57240
list indices must be integers or slices, not str
91978
list indices must be integers or slices, not str
35278
list indices must be integers or slices, not str
24459
list indices must be integers or slices, not str
152492
list indices must be integers or slices, not str
60609
list indices must be integers or slices, not str
15217
list indices must be integers or slices, not str
70192
list indices must be integers or slices, not str
74859
list indices must be integers or slices, not str
74860
list indices must be integers or slices, not str
31267
list indices must be integers or slices, not str
24948
list indices must be integers or slices, not str
59628
list indices must be integers or slices, not str
17565
list indices must be integers or slices, not str
16220
list indices must be integers or slices, not str
54314
list indi

list indices must be integers or slices, not str
11447
list indices must be integers or slices, not str
132405
list indices must be integers or slices, not str
78782
list indices must be integers or slices, not str
91995
list indices must be integers or slices, not str
62560
list indices must be integers or slices, not str
79440
list indices must be integers or slices, not str
11513
list indices must be integers or slices, not str
74071
list indices must be integers or slices, not str
71444
list indices must be integers or slices, not str
61487
list indices must be integers or slices, not str
150973
list indices must be integers or slices, not str
50481
list indices must be integers or slices, not str
62538
list indices must be integers or slices, not str
71004
list indices must be integers or slices, not str
6826
list indices must be integers or slices, not str
6729
list indices must be integers or slices, not str
62908
list indices must be integers or slices, not str
89581
list indic

list indices must be integers or slices, not str
2702
list indices must be integers or slices, not str
91332
list indices must be integers or slices, not str
72051
list indices must be integers or slices, not str
73090
list indices must be integers or slices, not str
74342
list indices must be integers or slices, not str
151242
list indices must be integers or slices, not str
73716
list indices must be integers or slices, not str
75125
list indices must be integers or slices, not str
72488
list indices must be integers or slices, not str
73718
list indices must be integers or slices, not str
7322
list indices must be integers or slices, not str
72965
list indices must be integers or slices, not str
72966
list indices must be integers or slices, not str
22132
list indices must be integers or slices, not str
75737
list indices must be integers or slices, not str
75736
list indices must be integers or slices, not str
91335
list indices must be integers or slices, not str
101615
list indic

list indices must be integers or slices, not str
10291
list indices must be integers or slices, not str
79644
list indices must be integers or slices, not str
32132
list indices must be integers or slices, not str
113279
list indices must be integers or slices, not str
59401
list indices must be integers or slices, not str
57074
list indices must be integers or slices, not str
87123
list indices must be integers or slices, not str
38821
list indices must be integers or slices, not str
30337
list indices must be integers or slices, not str
79234
list indices must be integers or slices, not str
56615
list indices must be integers or slices, not str
51481
list indices must be integers or slices, not str
85831
list indices must be integers or slices, not str
42869
list indices must be integers or slices, not str
41667
list indices must be integers or slices, not str
110376
list indices must be integers or slices, not str
150947
list indices must be integers or slices, not str
111536
list i

In [36]:
df['teams'] = df.apply(lambda row: collect('teams')(row['character_id']), axis=1)

list indices must be integers or slices, not str
1699
list indices must be integers or slices, not str
2048
list indices must be integers or slices, not str
2357
list indices must be integers or slices, not str
12663
list indices must be integers or slices, not str
5368
list indices must be integers or slices, not str
1808
list indices must be integers or slices, not str
5556
list indices must be integers or slices, not str
40431
list indices must be integers or slices, not str
3404
list indices must be integers or slices, not str
1791
list indices must be integers or slices, not str
1686
list indices must be integers or slices, not str
10451
list indices must be integers or slices, not str
5554
list indices must be integers or slices, not str
1688
list indices must be integers or slices, not str
6578
list indices must be integers or slices, not str
18276
list indices must be integers or slices, not str
7367
list indices must be integers or slices, not str
15809
list indices must be in

list indices must be integers or slices, not str
64958
list indices must be integers or slices, not str
21180
list indices must be integers or slices, not str
2532
list indices must be integers or slices, not str
17771
list indices must be integers or slices, not str
6904
list indices must be integers or slices, not str
54024
list indices must be integers or slices, not str
21183
list indices must be integers or slices, not str
34762
list indices must be integers or slices, not str
45081
list indices must be integers or slices, not str
29323
list indices must be integers or slices, not str
21406
list indices must be integers or slices, not str
21181
list indices must be integers or slices, not str
2550
list indices must be integers or slices, not str
23389
list indices must be integers or slices, not str
55024
list indices must be integers or slices, not str
53229
list indices must be integers or slices, not str
3633
list indices must be integers or slices, not str
37310
list indices m

list indices must be integers or slices, not str
53421
list indices must be integers or slices, not str
47831
list indices must be integers or slices, not str
92690
list indices must be integers or slices, not str
26635
list indices must be integers or slices, not str
33273
list indices must be integers or slices, not str
11290
list indices must be integers or slices, not str
19191
list indices must be integers or slices, not str
23344
list indices must be integers or slices, not str
76597
list indices must be integers or slices, not str
22134
list indices must be integers or slices, not str
11271
list indices must be integers or slices, not str
9549
list indices must be integers or slices, not str
61363
list indices must be integers or slices, not str
28491
list indices must be integers or slices, not str
19192
list indices must be integers or slices, not str
61389
list indices must be integers or slices, not str
50473
list indices must be integers or slices, not str
82953
list indice

list indices must be integers or slices, not str
67997
list indices must be integers or slices, not str
49741
list indices must be integers or slices, not str
15842
list indices must be integers or slices, not str
12189
list indices must be integers or slices, not str
50968
list indices must be integers or slices, not str
90781
list indices must be integers or slices, not str
29410
list indices must be integers or slices, not str
31123
list indices must be integers or slices, not str
56626
list indices must be integers or slices, not str
49685
list indices must be integers or slices, not str
79254
list indices must be integers or slices, not str
35743
list indices must be integers or slices, not str
61339
list indices must be integers or slices, not str
4605
list indices must be integers or slices, not str
34574
list indices must be integers or slices, not str
6846
list indices must be integers or slices, not str
88033
list indices must be integers or slices, not str
37305
list indices

list indices must be integers or slices, not str
61216
list indices must be integers or slices, not str
86071
list indices must be integers or slices, not str
48521
list indices must be integers or slices, not str
49481
list indices must be integers or slices, not str
56100
list indices must be integers or slices, not str
74957
list indices must be integers or slices, not str
45542
list indices must be integers or slices, not str
49701
list indices must be integers or slices, not str
49689
list indices must be integers or slices, not str
4851
list indices must be integers or slices, not str
56981
list indices must be integers or slices, not str
68103
list indices must be integers or slices, not str
3355
list indices must be integers or slices, not str
22636
list indices must be integers or slices, not str
19088
list indices must be integers or slices, not str
23273
list indices must be integers or slices, not str
21321
list indices must be integers or slices, not str
69766
list indices

...

list indices must be integers or slices, not str
10218
list indices must be integers or slices, not str
49912
list indices must be integers or slices, not str
59732
list indices must be integers or slices, not str
59803
list indices must be integers or slices, not str
87099
list indices must be integers or slices, not str
29035
list indices must be integers or slices, not str
26974
list indices must be integers or slices, not str
26880
list indices must be integers or slices, not str
3137
list indices must be integers or slices, not str
21594
list indices must be integers or slices, not str
77365
list indices must be integers or slices, not str
60675
list indices must be integers or slices, not str
27086
list indices must be integers or slices, not str
77170
list indices must be integers or slices, not str
60677
list indices must be integers or slices, not str
77366
list indices must be integers or slices, not str
78806
list indices must be integers or slices, not str
57363
list indice

list indices must be integers or slices, not str
15570
list indices must be integers or slices, not str
22935
list indices must be integers or slices, not str
22914
list indices must be integers or slices, not str
22991
list indices must be integers or slices, not str
22915
list indices must be integers or slices, not str
22916
list indices must be integers or slices, not str
65194
list indices must be integers or slices, not str
50873
list indices must be integers or slices, not str
10588
list indices must be integers or slices, not str
17646
list indices must be integers or slices, not str
51535
list indices must be integers or slices, not str
59744
list indices must be integers or slices, not str
22917
list indices must be integers or slices, not str
56641
list indices must be integers or slices, not str
57753
list indices must be integers or slices, not str
19371
list indices must be integers or slices, not str
22918
list indices must be integers or slices, not str
10668
list indic

list indices must be integers or slices, not str
73091
list indices must be integers or slices, not str
73122
list indices must be integers or slices, not str
85252
list indices must be integers or slices, not str
17371
list indices must be integers or slices, not str
73092
list indices must be integers or slices, not str
69567
list indices must be integers or slices, not str
76603
list indices must be integers or slices, not str
70307
list indices must be integers or slices, not str
78236
list indices must be integers or slices, not str
79266
list indices must be integers or slices, not str
112547
list indices must be integers or slices, not str
76907
list indices must be integers or slices, not str
76906
list indices must be integers or slices, not str
77358
list indices must be integers or slices, not str
80723
list indices must be integers or slices, not str
90470
list indices must be integers or slices, not str
74990
list indices must be integers or slices, not str
82626
list indi

list indices must be integers or slices, not str
36697
list indices must be integers or slices, not str
72276
list indices must be integers or slices, not str
85669
list indices must be integers or slices, not str
92479
list indices must be integers or slices, not str
77033
list indices must be integers or slices, not str
84155
list indices must be integers or slices, not str
93437
list indices must be integers or slices, not str
81155
list indices must be integers or slices, not str
77084
list indices must be integers or slices, not str
77085
list indices must be integers or slices, not str
76977
list indices must be integers or slices, not str
76976
list indices must be integers or slices, not str
58091
list indices must be integers or slices, not str
66780
list indices must be integers or slices, not str
83534
list indices must be integers or slices, not str
83469
list indices must be integers or slices, not str
150432
list indices must be integers or slices, not str
36085
list indi

list indices must be integers or slices, not str
32255
list indices must be integers or slices, not str
92178
list indices must be integers or slices, not str
60103
list indices must be integers or slices, not str
33239
list indices must be integers or slices, not str
48592
list indices must be integers or slices, not str
56793
list indices must be integers or slices, not str
83516
list indices must be integers or slices, not str
63886
list indices must be integers or slices, not str
52632
list indices must be integers or slices, not str
52633
list indices must be integers or slices, not str
32594
list indices must be integers or slices, not str
52624
list indices must be integers or slices, not str
79761
list indices must be integers or slices, not str
79516
list indices must be integers or slices, not str
145986
list indices must be integers or slices, not str
32985
list indices must be integers or slices, not str
14582
list indices must be integers or slices, not str
43186
list indi

In [37]:
df.to_json(path_or_buf='data/characters_teams_powers.json', orient='records')

Just writing down the dataframe, so its not collected every time.....

Read it, just to check what we have, and continue cleaning for our visualizations...

In [38]:
df_1 = pd.read_json('data/characters_teams_powers.json')

In [39]:
df_1.shape

(7408, 27)

In [40]:
df_1.head(2)

,aliases,align,alive,appearances,birth,character_id,character_name,count_of_issue_appearances,deck,description,eye,first appearance,first_appeared_in_issue,gsm,hair,identity_type,image,join_name,origin,page_id,powers,publisher,real_name,sex,teams,urlslug,year
0,Bruce Wayne\r\nThe Caped Crusader\r\nThe Dark ...,Good Characters,Living Characters,3093.0,"Feb 19, 1972",1699,Batman,16603,"Bruce Wayne, who witnessed the murder of his m...","<h2>Current Events</h2><ul><li><a href=""https:...",Blue Eyes,May-39,May-39,None,Black Hair,Secret Identity,https://comicvine.gamespot.com/api/image/scale...,batman_may_39,Human,1422,"[Agility, Stamina, Intellect, Weapon Master, I...",DC Comics,Bruce Wayne,Male Characters,None,\/wiki\/Batman_(Bruce_Wayne),1939.0
1,Diana Prince\r\nDiana of Themyscira\r\nPrinces...,Good Characters,Living Characters,1231.0,"Mar 21, 1881",2048,Wonder Woman,6575,"The princess of the Amazons, armed with powers...",<h2><strong>Origin</strong></h2><figure data-a...,Blue Eyes,Dec-41,Dec-41,None,Black Hair,Public Identity,https://comicvine.gamespot.com/api/image/scale...,wonder_woman_dec_41,Other,1448,"[Flight, Super Strength, Super Speed, Agility,...",DC Comics,Diana of Themyscira,Female Characters,None,\/wiki\/Wonder_Woman_(Diana_Prince),1941.0


In [41]:
df_1[df_1.teams.notna()].shape

(1764, 27)

In [42]:
df_1[df_1.powers.notna()].shape

(1893, 27)

We can see that there is 1592 characters that are within a team. Also, there is 1620 characters with some kind of power.

In [43]:
df_1 = df_1.drop(columns=['description', 'first appearance', 'urlslug'])

In [44]:
def spliting_by_separator(x):
    if isinstance(x, Iterable):
        if type(x) == str:
            return x
        else:
            return '; '.join(x)
    else:
        return np.nan

In [45]:
def converting_float_to_int(x):
    if str(x).endswith(".0"):
        return str(x)[:-2]
    else:
        return x

In [46]:
from collections.abc import Iterable

df_1['aliases'] = df_1.apply(lambda row: '; '.join(str(row['aliases']).split('\r\n')[:5]), axis=1)
df_1['appearances'] = df_1.apply(lambda row: converting_float_to_int(row['appearances']), axis=1)
df_1['year'] = df_1.apply(lambda row: converting_float_to_int(row['year']), axis=1)
df_1['real_name'] = df_1.apply(lambda row: '; '.join(str(row['real_name']).split('\r\n')[:5]), axis=1)
df_1['powers'] = df_1.apply(lambda row: spliting_by_separator(row['powers']), axis=1)
df_1['teams'] = df_1.apply(lambda row: spliting_by_separator(row['teams']), axis=1)

In [47]:
df_1.shape

(7408, 24)

In [48]:
df_1.head(5)

,aliases,align,alive,appearances,birth,character_id,character_name,count_of_issue_appearances,deck,eye,first_appeared_in_issue,gsm,hair,identity_type,image,join_name,origin,page_id,powers,publisher,real_name,sex,teams,year
0,Bruce Wayne; The Caped Crusader; The Dark Knig...,Good Characters,Living Characters,3093,"Feb 19, 1972",1699,Batman,16603,"Bruce Wayne, who witnessed the murder of his m...",Blue Eyes,May-39,None,Black Hair,Secret Identity,https://comicvine.gamespot.com/api/image/scale...,batman_may_39,Human,1422,Agility; Stamina; Intellect; Weapon Master; In...,DC Comics,Bruce Wayne,Male Characters,NaN,1939
1,Diana Prince; Diana of Themyscira; Princess Di...,Good Characters,Living Characters,1231,"Mar 21, 1881",2048,Wonder Woman,6575,"The princess of the Amazons, armed with powers...",Blue Eyes,Dec-41,None,Black Hair,Public Identity,https://comicvine.gamespot.com/api/image/scale...,wonder_woman_dec_41,Other,1448,Flight; Super Strength; Super Speed; Agility; ...,DC Comics,Diana of Themyscira,Female Characters,NaN,1941
2,Orin; Aquafresh; Aquaman; King Arthur; King of...,Good Characters,Living Characters,1121,"Jan 29, 1977",2357,Aquaman,3505,The son of an Atlantean queen and a lighthouse...,Blue Eyes,Nov-41,None,Blond Hair,Public Identity,https://comicvine.gamespot.com/api/image/scale...,aquaman_nov_41,Other,1486,Super Strength; Super Speed; Agility; Stamina;...,DC Comics,Arthur Curry,Male Characters,NaN,1941
3,Alan Wellington Scott\nAlan Ladd Scott\nSentin...,Good Characters,Deceased Characters,969,None,12663,Alan Scott,1666,"Alan Scott, the bearer of the mystical Starhea...",Blue Eyes,Jul-40,None,Blond Hair,Secret Identity,https://comicvine.gamespot.com/api/image/scale...,alan_scott_jul_40,Human,1459,Flight; Super Speed; Intellect; Psychic; Force...,DC Comics,Alan Ladd Wellington Scott,Male Characters,NaN,1940
4,Amy Beddoes; Batgirl; Oracle; Babs; Nightwing,Good Characters,Living Characters,951,"Sep 23, 1990",5368,Barbara Gordon,2864,Barbara Gordon was the first modern age Batgir...,Blue Eyes,Jan-67,None,Red Hair,Secret Identity,https://comicvine.gamespot.com/api/image/scale...,barbara_gordon_jan_67,Human,1905,Agility; Stamina; Intellect; Psychic; Weapon M...,DC Comics,Barbara Gordon,Female Characters,NaN,1967


In [49]:
import pandas_profiling
profile = pandas_profiling.ProfileReport(df_1)
profile.to_file(outputfile="reports/ReportVisualizationDataset.html")

In [50]:
df_1.to_json(path_or_buf='data/characters_teams_powers_cleaned.json', orient='records')

In [51]:
df_1.to_csv(path_or_buf='data/characters_teams_powers_cleaned.csv')